In [1]:
import os 
import tqdm
import math
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from numpy import count_nonzero

dir_ = '../data/'
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'

In [2]:
predictions = pd.read_pickle(os.path.join(dir_, 'prediction_SVD_user_top_N_' + file_name[:-3] + 'pkl'))

In [3]:
predictions

,uid,tid,rating
0,0.0,0.0,3.270248
1,0.0,8.0,0.250000
2,0.0,9.0,2.969405
3,0.0,14.0,2.500000
4,0.0,16.0,0.250000
...,...,...,...
3072115,219.0,54914.0,0.250000
3072116,219.0,54940.0,0.250000
3072117,219.0,54941.0,0.250000
3072118,219.0,54947.0,2.250000


In [4]:
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))

In [5]:
test

,uid,tid,rating
481832,90,5374,0.25
1247457,214,35528,1.25
968433,172,29367,4.25
25524,4,11632,4.25
60178,8,19012,0.25
...,...,...,...
587045,115,27784,3.25
309534,65,12495,0.25
660480,130,20821,0.25
373255,73,27444,2.75


In [6]:
num_user = len(predictions['uid'].unique())
num_user

220

In [7]:
# Personal Recommendation
n = 20
top_n_lists = []

for i in tqdm.tqdm_notebook(range(num_user)):
    prediction = predictions[predictions['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
    top_n_lists.append(top_n_list)

In [8]:
evaluation = []
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/num_user)


0.10165876777251191


In [24]:
#############################################
#                                           #
#          Group Recommendation             #
#                                           #
#############################################

In [55]:
#############################################
#                                           #
#          Group Recommendation (2)         #
#                                           #
#############################################

In [7]:
groups = np.loadtxt(os.path.join(dir_, 'groups_2_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([32., 78.])

In [8]:
def max_satisfaction(df):  
    return max(df['rating_u1'], df['rating_u2'])

def least_misery(df):
    return min(df['rating_u1'], df['rating_u2'])

In [9]:
# Prediction list of each group (2)
n=4
top_n_lists_g = []
    
for group in tqdm.tqdm_notebook(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]

    m = pd.merge(u1, u2, on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = m[['tid', 'rating_u1','rating_u2']]
    
#   Average
    m['rating'] = (m['rating_u1'] + m['rating_u2'])/2

#   Max Satisfaction
#     m['rating'] = m.apply(max_satisfaction, axis = 1)

#   Least Misery
#     m['rating'] = m.apply(least_misery, axis = 1)
    
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)

In [10]:
evaluation = []
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]

        s = len(t1) + len(t2)
#         if s > 0:
#             precision += 1
        precision += math.log(s+1,10)/math.log(len(group)+1,10)
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/(num_user//group_size))


0.056410640608768416


In [11]:
#############################################
#                                           #
#          Group Recommendation (4)         #
#                                           #
#############################################

In [12]:
groups = np.loadtxt(os.path.join(dir_, 'groups_4_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([ 16.,  69., 111., 129.])

In [13]:
def max_satisfaction(df):  
    return max(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'])

def least_misery(df):
    return min(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'])

In [14]:
# Prediction list of each group (4)
n=4
top_n_lists_g = []
    
for group in tqdm.tqdm_notebook(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]
    u3 = predictions[predictions['uid'] == group[2]]
    u4 = predictions[predictions['uid'] == group[3]]
    
    m = pd.merge(u1, u2, on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = pd.merge(m,  u3, on=['tid'], how='inner', suffixes=['','_u3'])
    m = pd.merge(m,  u4, on=['tid'], how='inner', suffixes=['_u3','_u4'])
    m = m[['tid', 'rating_u1','rating_u2','rating_u3','rating_u4']]
#     Average
    m['rating'] = (m['rating_u1'] + m['rating_u2'] + m['rating_u3'] + m['rating_u4'])/4

#   Max Satisfaction
#     m['rating'] = m.apply(max_satisfaction, axis = 1)

#   Least Misery
#     m['rating'] = m.apply(least_misery, axis = 1)
    
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)

In [15]:
evaluation = []
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4)
#         if s > 0:
#             precision += 1
        precision += math.log(s+1,10)/math.log(len(group)+1,10)
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/(num_user//group_size))


0.08875822974002853


In [ ]:
#############################################
#                                           #
#          Group Recommendation (5)         #
#                                           #
#############################################

In [16]:
groups = np.loadtxt(os.path.join(dir_, 'groups_5_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([ 11.,  23., 107., 122., 141.])

In [17]:
def max_satisfaction(df):  
    return max(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'], df['rating_u5'])

def least_misery(df):
    return min(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'], df['rating_u5'])

In [18]:
# Prediction list of each group (5)
n=4
top_n_lists_g = []
    
for group in tqdm.tqdm_notebook(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]
    u3 = predictions[predictions['uid'] == group[2]]
    u4 = predictions[predictions['uid'] == group[3]]
    u5 = predictions[predictions['uid'] == group[4]]

    m = pd.merge(u1, u2, on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = pd.merge(m,  u3, on=['tid'], how='inner', suffixes=['','_u3'])
    m = pd.merge(m,  u4, on=['tid'], how='inner', suffixes=['','_u4'])
    m = pd.merge(m,  u5, on=['tid'], how='inner', suffixes=['_u3','_u5'])
    m = m[['tid', 'rating_u1','rating_u2','rating_u3','rating_u4','rating_u5']]
#     Average
    m['rating'] = (m['rating_u1'] + m['rating_u2'] + m['rating_u3'] + m['rating_u4'] + m['rating_u5'])/5
    
#     Max Satisfaction
#     m['rating'] = m.apply(max_satisfaction, axis = 1)
    
#     Least Misery
#     m['rating'] = m.apply(least_misery, axis = 1)
    
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)

In [19]:
evaluation = []
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5)
#         if s > 0:
#             precision += 1
        precision += math.log(s+1,10)/math.log(len(group)+1,10)
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/(num_user//group_size))


0.10791157400153856


In [ ]:
#############################################
#                                           #
#          Group Recommendation (10)        #
#                                           #
#############################################

In [20]:
# np.savetxt(os.path.join(dir_, 'groups_10_' + file_name[:-3] + 'csv'), groups, delimiter=",")
groups = np.loadtxt(os.path.join(dir_, 'groups_10_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([ 17.,  58.,  73., 104., 112., 164., 207., 214., 218., 219.])

In [21]:
def max_satisfaction(df):  
    return max(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'], df['rating_u5'], df['rating_u6'], df['rating_u7'], df['rating_u8'], df['rating_u9'], df['rating_u10'])

def least_misery(df):  
    return max(df['rating_u1'], df['rating_u2'], df['rating_u3'], df['rating_u4'], df['rating_u5'], df['rating_u6'], df['rating_u7'], df['rating_u8'], df['rating_u9'], df['rating_u10'])

In [22]:
# Prediction list of each group (10)
n=4
top_n_lists_g = []
    
for group in tqdm.tqdm_notebook(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]
    u3 = predictions[predictions['uid'] == group[2]]
    u4 = predictions[predictions['uid'] == group[3]]
    u5 = predictions[predictions['uid'] == group[4]]
    u6 = predictions[predictions['uid'] == group[5]]
    u7 = predictions[predictions['uid'] == group[6]]
    u8 = predictions[predictions['uid'] == group[7]]
    u9 = predictions[predictions['uid'] == group[8]]
    u10 = predictions[predictions['uid'] == group[9]]

    m = pd.merge(u1, u2,  on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = pd.merge(m,  u3,  on=['tid'], how='inner', suffixes=['','_u3'])
    m = pd.merge(m,  u4,  on=['tid'], how='inner', suffixes=['','_u4'])
    m = pd.merge(m,  u5,  on=['tid'], how='inner', suffixes=['','_u5'])
    m = pd.merge(m,  u6,  on=['tid'], how='inner', suffixes=['','_u6'])
    m = pd.merge(m,  u7,  on=['tid'], how='inner', suffixes=['','_u7'])
    m = pd.merge(m,  u8,  on=['tid'], how='inner', suffixes=['','_u8'])
    m = pd.merge(m,  u9,  on=['tid'], how='inner', suffixes=['','_u9'])
    m = pd.merge(m,  u10, on=['tid'], how='inner', suffixes=['_u3','_u10'])
    m = m[['tid', 'rating_u1','rating_u2','rating_u3','rating_u4','rating_u5', 'rating_u6','rating_u7','rating_u8','rating_u9','rating_u10']]

#     Average
    m['rating'] = (m['rating_u1'] + m['rating_u2'] + m['rating_u3'] + m['rating_u4'] + m['rating_u5'] + m['rating_u6'] + m['rating_u7'] + m['rating_u8'] + m['rating_u9'] + + m['rating_u10'])/10
    
#     Max Satisfaction
#     m['rating'] = m.apply(max_satisfaction, axis = 1)
    
#     Least Misery
#     m['rating'] = m.apply(least_misery, axis = 1)
    
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)

In [23]:
evaluation = []
satisfication = 0
for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        t6 = test[test['uid'] == group[5]]
        t7 = test[test['uid'] == group[6]]
        t8 = test[test['uid'] == group[7]]
        t9 = test[test['uid'] == group[8]]
        t10 = test[test['uid'] == group[9]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        t6 = t6[t6['tid']==j]
        t7 = t7[t7['tid']==j]
        t8 = t8[t8['tid']==j]
        t9 = t9[t9['tid']==j]
        t10 = t10[t10['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5) + len(t6) + len(t7) + len(t8) + len(t9) + len(t10)
#         if s > 0:
#             precision += 1
        precision += math.log(s+1,10)/math.log(len(group)+1,10)
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/(num_user//group_size))


0.24141876141292426
